In [12]:
import torch
import numpy as np
from lib.networks import anime_full_encoder, anime_eye_encoder
from lib.data import Data
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import os

In [2]:
lr_full = 0.0001
lr_eye = 0.0002
beta1=0.5
beta2=0.999

In [3]:
full_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

eye_transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

full_path = "C:/Users/teejs/Documents/Dataset/Anime Data/anime_face_large_dif_pos"
leye_path = "left_eye"
reye_path = "right_eye"
train_loader_full = Data(full_path, full_transform, shuffle=True)
train_loader_leye = Data(leye_path, eye_transform)
train_loader_reye = Data(reye_path, eye_transform)

In [4]:
# leye_image = Image.open("left_eye/42001-1-0.png")

# leye_image = transform(leye_image)
# leye_image = leye_image.unsqueeze(0)
# leye_image.size()

In [5]:
L1_loss = nn.L1Loss()

In [6]:
model_full = anime_full_encoder(3)
model_leye = anime_eye_encoder(3)
model_reye = anime_eye_encoder(3)
all_models = {
    'model_full.pth':model_full,
    'model_leye.pth':model_leye,
    'model_reye.pth':model_reye
}

In [7]:
optimizer_full = optim.Adam(model_full.parameters(), lr=lr_full, betas=(beta1, beta2))
optimizer_leye = optim.Adam(model_leye.parameters(), lr=lr_eye, betas=(beta1, beta2))
optimizer_reye = optim.Adam(model_reye.parameters(), lr=lr_eye, betas=(beta1, beta2))

In [8]:
loss = 100
count=0

In [13]:
def save_models(models, folder):
    for k, v in models.items():
        torch.save(v.state_dict(), os.path.join(folder, k))

In [10]:
def load_models(models, folder):
    for k, v in models.items():
        v.load_state_dict(torch.load(os.path.join(folder, k)))

In [ ]:
while(loss>0.01):
    try:
        leye_img,leye_img_name = train_loader_leye.next()
        full_img = train_loader_full.get(leye_img_name)
        reye_img = train_loader_reye.get(leye_img_name)

    #     full_img,full_img_name = train_loader_full.next()
    #     leye_img = train_loader_leye.get(full_img_name)
    #     reye_img = train_loader_reye.get(full_img_name) 
        if type(full_img) == type(None) or type(reye_img) == type(None):
            continue

        optimizer_full.zero_grad()
        optimizer_leye.zero_grad()
        optimizer_reye.zero_grad()   

        full_recon, full_result_l, full_result_r = model_full(full_img)
        leye_recon, lresult = model_leye(leye_img)
        reye_recon, rresult = model_reye(reye_img)
        recon_loss = L1_loss(full_recon,full_img)+L1_loss(leye_recon,leye_img)+L1_loss(reye_recon,reye_img)
        latent_loss = L1_loss(full_result_l,lresult) + L1_loss(full_result_r, rresult)
        loss = recon_loss+latent_loss
        loss.backward()

        optimizer_full.step()
        optimizer_leye.step()
        optimizer_reye.step()
    except:
        continue
    
    if count%100==0:
        print(loss)
    if count%1000==0:
        save_models(all_models,'model')
    count+=1
    

tensor(15.1434, grad_fn=<ThAddBackward>)
tensor(16.0883, grad_fn=<ThAddBackward>)
tensor(15.8783, grad_fn=<ThAddBackward>)
tensor(12.0932, grad_fn=<ThAddBackward>)
tensor(15.8468, grad_fn=<ThAddBackward>)
tensor(12.4800, grad_fn=<ThAddBackward>)
tensor(14.1574, grad_fn=<ThAddBackward>)
tensor(17.9641, grad_fn=<ThAddBackward>)
tensor(13.4033, grad_fn=<ThAddBackward>)
tensor(11.8100, grad_fn=<ThAddBackward>)
tensor(13.7460, grad_fn=<ThAddBackward>)
tensor(13.4672, grad_fn=<ThAddBackward>)
tensor(15.8871, grad_fn=<ThAddBackward>)
tensor(12.6656, grad_fn=<ThAddBackward>)
tensor(16.7386, grad_fn=<ThAddBackward>)
tensor(13.0086, grad_fn=<ThAddBackward>)
tensor(13.8166, grad_fn=<ThAddBackward>)
tensor(10.6128, grad_fn=<ThAddBackward>)
tensor(11.8560, grad_fn=<ThAddBackward>)
tensor(11.7513, grad_fn=<ThAddBackward>)
tensor(12.3832, grad_fn=<ThAddBackward>)
tensor(12.3086, grad_fn=<ThAddBackward>)
tensor(13.8810, grad_fn=<ThAddBackward>)
tensor(12.8832, grad_fn=<ThAddBackward>)
tensor(13.3786, 

tensor(9.2749, grad_fn=<ThAddBackward>)
tensor(10.2589, grad_fn=<ThAddBackward>)
tensor(9.5552, grad_fn=<ThAddBackward>)
tensor(9.3089, grad_fn=<ThAddBackward>)
tensor(10.2122, grad_fn=<ThAddBackward>)
tensor(9.2142, grad_fn=<ThAddBackward>)
tensor(9.5159, grad_fn=<ThAddBackward>)
tensor(9.0991, grad_fn=<ThAddBackward>)
tensor(11.0316, grad_fn=<ThAddBackward>)
tensor(9.6293, grad_fn=<ThAddBackward>)
tensor(10.2158, grad_fn=<ThAddBackward>)
tensor(8.8631, grad_fn=<ThAddBackward>)
tensor(10.3991, grad_fn=<ThAddBackward>)
tensor(9.9826, grad_fn=<ThAddBackward>)
tensor(9.7475, grad_fn=<ThAddBackward>)
tensor(9.7449, grad_fn=<ThAddBackward>)
tensor(9.8746, grad_fn=<ThAddBackward>)
tensor(10.4144, grad_fn=<ThAddBackward>)
tensor(9.8219, grad_fn=<ThAddBackward>)
tensor(10.6362, grad_fn=<ThAddBackward>)
tensor(10.2660, grad_fn=<ThAddBackward>)
tensor(10.2970, grad_fn=<ThAddBackward>)
tensor(9.8369, grad_fn=<ThAddBackward>)
tensor(9.9935, grad_fn=<ThAddBackward>)
tensor(9.3279, grad_fn=<ThAddBa

In [ ]:
save_models(all_models,'model')